In [ ]:
# Import necessary libraries
import numpy as np

# Generate a 3D lattice with random spins for a 5x5x5 system
N_3D = 5

# Initialize the lattice with random spins
init_random_3D = np.random.random((N_3D, N_3D, N_3D))
lattice_3D = np.zeros((N_3D, N_3D, N_3D))
lattice_3D[init_random_3D >= 0.5] = 1
lattice_3D[init_random_3D < 0.5] = -1

# Save the lattice to a file
np.save("lattice.py", lattice_3D)

lattice_3D  # print the generated lattice to confirm its creation

In [ ]:
# Import necessary libraries
from scipy.ndimage import convolve, generate_binary_structure
from numba import njit
import os

# Define a function to calculate the energy of a given 3D lattice configuration
def get_energy_3D(lattice):
    # Applies the nearest neighbours summation
    kern = generate_binary_structure(3, 1) 
    kern[1, 1, 1] = False
    arr = -lattice * convolve(lattice, kern, mode='constant', cval=0)
    return arr.sum()

# Correcting the return type for the metropolis_3D_single_step function
@njit("Tuple((f8[:,:,:], f8, f8))(f8[:,:,:], f8, f8)", nopython=True, nogil=True)
def metropolis_3D_single_step(spin_arr, BJ, energy):
    spin_arr = spin_arr.copy()

    # Pick random point on array and flip spin
    x = np.random.randint(0, N_3D)
    y = np.random.randint(0, N_3D)
    z = np.random.randint(0, N_3D)
    spin_i = spin_arr[x, y, z] #initial spin
    spin_f = spin_i*-1 #proposed spin flip

    # Compute change in energy
    E_i = 0
    E_f = 0
    for dx, dy, dz in [(-1, 0, 0), (1, 0, 0), (0, -1, 0), (0, 1, 0), (0, 0, -1), (0, 0, 1)]:
        nx, ny, nz = x + dx, y + dy, z + dz
        if 0 <= nx < N_3D and 0 <= ny < N_3D and 0 <= nz < N_3D:
            E_i += -spin_i*spin_arr[nx, ny, nz]
            E_f += -spin_f*spin_arr[nx, ny, nz]

    # Change state with designated probabilities
    dE = E_f-E_i
    if (dE>0)*(np.random.random() < np.exp(-BJ*dE)):
        spin_arr[x, y, z]=spin_f
        energy += dE
    elif dE<=0:
        spin_arr[x, y, z]=spin_f
        energy += dE

    net_spin = spin_arr.sum()
    return spin_arr, net_spin, energy

# Run the Metropolis algorithm for a specified number of steps, and save the spin configuration after each step
def run_metropolis_3D(lattice, steps, BJ, energy):
    spins = np.zeros(steps)
    energies = np.zeros(steps)
    for t in range(steps):
        lattice, spins[t], energies[t] = metropolis_3D_single_step(lattice, BJ, energy)
        np.save(f"spin_config_{t}.npy", lattice)
    return spins, energies

# Test the functions with the initial 3D lattice
spins_3D, energies_3D = run_metropolis_3D(lattice_3D, 100, 0.2, get_energy_3D(lattice_3D))

spins_3D, energies_3D  # print the resulting spins and energies to confirm the function's operation


In [ ]:
# Load the spin configuration after the 50th step
spin_config_50 = np.load("spin_config_50.npy")

# Print the spin configuration
spin_config_50

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define a function to visualize a single 3D lattice configuration
def visualize_lattice_3D(lattice, step):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Create arrays to hold the coordinates and colors of the atoms
    x, y, z = np.indices(lattice.shape)
    c = np.where(lattice > 0, 'r', 'b')

    # Plot the atoms as a scatter plot and the spins as a quiver plot
    ax.scatter(x, y, z, c=c.flatten(), s=100)
    ax.quiver(x, y, z, 0, 0, lattice, color=c.flatten())

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Spin configuration at step {step}')
    plt.show()

# Load the spin configuration after the 50th step
spin_config_50 = np.load("spin_config_50.npy")

# Visualize the spin configuration
visualize_lattice_3D(spin_config_50, 50)

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Define a function to visualize a single 3D lattice configuration
def visualize_lattice_3D(lattice, step):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Create arrays to hold the coordinates and colors of the atoms
    x, y, z = np.indices(lattice.shape)
    c = np.where(lattice > 0, 'r', 'b')

    # Plot the atoms as a scatter plot and the spins as a quiver plot
    ax.scatter(x, y, z, c=c.flatten(), s=100)
    ax.quiver(x, y, z, 0, 0, lattice, color=c.flatten())

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Spin configuration at step {step}')
    plt.savefig(f"spin_config_{step}.png")

# Load the spin configuration after the 50th step
# spin_config_50 = np.load("/mnt/data/spin_config_50.npy")

steps = 100

for t in range(steps):
    spin_config_n = np.load(f"spin_config_{t}.npy")
    visualize_lattice_3D(spin_config_n, t)
